In [17]:
import numpy as np
import matplotlib.pyplot as plt
import random
from numba import njit
from sklearn.cluster import KMeans

N=1024
var_J=1

@njit
def initial_lattice(N):
    M=np.ones((N))
    l=np.array([1,-1])
    for i in range(N):
            M[i]=np.random.choice(l)
    return(M)

@njit
def intraction(N,mean_J,var_J):
    J_matrix=np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            if i<j :
                J_matrix[i][j]=random.gauss(mean_J/N,var_J/np.sqrt(N))
            else :
                J_matrix[i][j]=0
    J_matrix=J_matrix+np.transpose(J_matrix)  
    return(J_matrix)

@njit
def energy_local(M,J_matrix,i):
    H=np.sum(-J_matrix[i]*M[i]*M)
    return(H)

@njit
def energy_total(M,J_matrix,N):
    E_t=0
    for i in range(N):
        E_t=E_t+energy_local(M,J_matrix,i)
    E_t=E_t/2
    return(E_t)


@njit
def trial(J_matrix,N,T): 
    field_list=np.zeros(N)
    M=initial_lattice(N)
    E_t=energy_total(M,J_matrix,N)
    for trial in range(int(np.sqrt(N**3))):
        random_i=random.randrange(N)                
        M[random_i]=M[random_i]*(-1)
        delta_E=2*energy_local(M,J_matrix,random_i)
        if delta_E>0:
            p=np.exp(-delta_E/T)
            coin=random.random()
            if coin>p :
                M[random_i]=M[random_i]*(-1)
            else :
                E_t=E_t+delta_E
        else :
            E_t=E_t+delta_E
    
    for i in range(N):
        field=np.sum(-J_matrix[i]*M)
        field_list[i]=field

    return(field_list)

#spin glass
l_feature_SG=[]
for mean_J in np.arange(0.000001,0.010001,0.001):
    for T in np.arange(0.000001,0.010001,0.001):
        l=[]
        J_matrix=intraction(N,mean_J,var_J)
        field_time_series=trial(J_matrix,N,T)
        l.append(abs(np.mean(field_time_series)))
        l.append(np.var(field_time_series))
        l_feature_SG.append(l)

#paramagnetic  
l_feature_P=[]
for mean_J in np.arange(0.000001,0.010001,0.001):
    for T in np.arange(1000,1000.01,0.001):
        l=[]
        J_matrix=intraction(N,mean_J,var_J)
        field_time_series=trial(J_matrix,N,T)
        l.append(abs(np.mean(field_time_series)))
        l.append(np.var(field_time_series))
        l_feature_P.append(l)

#ferro    
l_feature_F=[]
for mean_J in np.arange(1000,1000.01,0.001):
    for T in np.arange(0.000001,0.010001,0.001):
        l=[]
        J_matrix=intraction(N,mean_J,var_J)
        field_time_series=trial(J_matrix,N,T)
        l.append(abs(np.mean(field_time_series))/mean_J)
        l.append(np.var(field_time_series))
        l_feature_F.append(l)


In [21]:
def bootstrap_ci(data, statfunc=np.mean, n_boot=2000, ci=0.95, rng=None):

    rng = np.random.default_rng(rng)
    data = np.asarray(data)
    n = len(data)

    boots = np.empty((n_boot, data.shape[1] if data.ndim > 1 else 1))
    for i in range(n_boot):
        sample_idx = rng.integers(0, n, n) 
        sample = data[sample_idx]
        boots[i] = statfunc(sample, axis=0)

    center = statfunc(data, axis=0)
    lo = np.percentile(boots, 100 * (1 - ci) / 2, axis=0)
    hi = np.percentile(boots, 100 * (1 + ci) / 2, axis=0)
    return center, (lo, hi), boots

mean_centroid_SG, (lo_ci_SG, hi_ci_SG), boots = bootstrap_ci(l_feature_SG)
print("Mean centroid_SG:", mean_centroid_SG)
print("95% CI lower:", lo_ci_SG)
print("95% CI upper:", hi_ci_SG)

mean_centroid_P, (lo_ci_P, hi_ci_P), boots = bootstrap_ci(l_feature_P)
print("Mean centroid_SG:", mean_centroid_P)
print("95% CI lower:", lo_ci_P)
print("95% CI upper:", hi_ci_P)

mean_centroid_F, (lo_ci_F, hi_ci_F), boots = bootstrap_ci(l_feature_F)
print("Mean centroid_SG:", mean_centroid_F)
print("95% CI lower:", lo_ci_F)
print("95% CI upper:", hi_ci_F)

Mean centroid_SG: [0.03600808 2.43891119]
95% CI lower: [0.03056039 2.42340552]
95% CI upper: [0.04129064 2.45288292]
Mean centroid_SG: [0.02763012 0.98456576]
95% CI lower: [0.02384838 0.9762797 ]
95% CI upper: [0.031828   0.99310813]
Mean centroid_SG: [0.99902822 0.99962293]
95% CI lower: [0.99901896 0.99174052]
95% CI upper: [0.99903759 1.00706607]
